# Applied Deep Learning Tutorial 
# Recurrent Neural Networks (RNNs) for Text Generation

## Introduction
In this tutorial, you will attempt to implement a recurrent neural network for text generation based on a text dataset from Shakespeare / Goethe. The tutorial has been inspired by the original [tensorflow tutorials](www.tensorflow.org) and Andrej Karpathies work and thus based on numpy only. 

<img src="graphics/GoetheSchillerWeimar.jpg" width="700"><br>
<center> Fig. 1: The german poets Goethe and Schiller in Weimar. Image from [pixabay](https://pixabay.com/de/photos/weimar-goethe-schiller-denkmal-806851/) </center>


## Core Idea 

In this tutorial we will focus on a text generation approach based on a character-based RNN. We will work with a dataset similar to the dataset of Shakespeare's writing provided by Andrej Karpathy's and first used in his [The Unreasonable Effectiveness of Recurrent Neural Networks](http://karpathy.github.io/2015/05/21/rnn-effectiveness/). Given a sequence of characters from this data ("Mephist"), train a model to predict the next character in the sequence ("o"). Longer sequences of text can be generated by calling the model repeatedly, or in other words recurrent.


In [1]:
import numpy as np
import string

## Scraping Text Data

First we need to scrape data. A good source to scrape is [Project Gutenberg](https://www.gutenberg.org/).
Since there are legal struggles with project gutenberg we will use Faust 1 by Goethe, scraped from [wikisource](https://de.wikisource.org/wiki/Faust_-_Der_Tragödie_erster_Teil). Or shakespeare already scraped for us by Google.


In [2]:
# Download the data
#path_to_file = tf.keras.utils.get_file('shakespeare.txt', 'https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt')

# Link to Goethe data
path_to_file = 'data/Faust1_Goethe.txt'



In [3]:
# Read, then decode for py2 compat.
text = open(path_to_file, 'rb').read().decode(encoding='utf-8')

# length of text is the number of characters in it
print ('Length of text: {} characters'.format(len(text)))

Length of text: 199943 characters


In [4]:
# Look at the first 1000 characters
print(text[:1000])

Faust - Der Tragödie erster Teil
[1]
Faust.
Eine Tragödie.
von
Goethe.
Tübingen.
in der J. G. Cotta’schen Buchhandlung.
1808.
[2] WS: Bibliotheksstempel und Signatur
[3]
Zueignung.
[4]
[5]
Ihr naht euch wieder, schwankende Gestalten!
Die früh sich einst dem trüben Blick gezeigt.
Versuch’ ich wohl euch diesmal fest zu halten?
Fühl’ ich mein Herz noch jenem Wahn geneigt?
Ihr drängt euch zu! nun gut, so mögt ihr walten,
Wie ihr aus Dunst und Nebel um mich steigt;
Mein Busen fühlt sich jugendlich erschüttert
Vom Zauberhauch der euren Zug umwittert.
Ihr bringt mit euch die Bilder froher Tage,
Und manche liebe Schatten steigen auf;
Gleich einer alten, halbverklungnen Sage,
Kommt erste Lieb’ und Freundschaft mit herauf;
Der Schmerz wird neu, es wiederholt die Klage
Des Lebens labyrinthisch irren Lauf,
Und nennt die Guten, die, um schöne Stunden
Vom Glück getäuscht, vor mir hinweggeschwunden.
[6]
Sie hören nicht die folgenden Gesänge,
Die Seelen, denen ich die e


In [5]:
# The unique characters in the file
vocab = sorted(set(text))
print ('{} unique characters'.format(len(vocab)))

# Print the vocabulary
print(vocab)


82 unique characters
['\n', '\r', ' ', '!', ',', '-', '.', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Z', '[', ']', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', 'ß', 'ä', 'ö', 'ü', '–', '’', '“', '”', '„']


In [6]:
# Delete unwanted characters
import re
char_list = ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '\[', ']']
text = re.sub("|".join(char_list), "", text)

# The unique characters in the file
vocab = sorted(set(text))
print ('{} unique characters'.format(len(vocab)))

# Print the vocabulary
print(vocab)

# Look at the first 1000 characters
print(text[:1000])

70 unique characters
['\n', '\r', ' ', '!', ',', '-', '.', ':', ';', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Z', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', 'ß', 'ä', 'ö', 'ü', '–', '’', '“', '”', '„']
Faust - Der Tragödie erster Teil

Faust.
Eine Tragödie.
von
Goethe.
Tübingen.
in der J. G. Cotta’schen Buchhandlung.
.
 WS: Bibliotheksstempel und Signatur

Zueignung.


Ihr naht euch wieder, schwankende Gestalten!
Die früh sich einst dem trüben Blick gezeigt.
Versuch’ ich wohl euch diesmal fest zu halten?
Fühl’ ich mein Herz noch jenem Wahn geneigt?
Ihr drängt euch zu! nun gut, so mögt ihr walten,
Wie ihr aus Dunst und Nebel um mich steigt;
Mein Busen fühlt sich jugendlich erschüttert
Vom Zauberhauch der euren Zug umwittert.
Ihr bringt mit euch die Bilder froher Tage,
Und manche liebe Schatten steigen auf;
Gleich eine

## Vectorize the characters
Before training the strings need to be vectorized, meaning we need to bring them in a numerical representation, for our neural network to be able to work with. 



In [7]:
# Creating a mapping from unique characters to indices
char_to_ix = {u:i for i, u in enumerate(vocab)}
ix_to_char = np.array(vocab)

text_as_int = np.array([char_to_ix[c] for c in text])

# Visualize the mapping
print('{')
for char,_ in zip(char_to_ix, range(20)):
    print('  {:4s}: {:3d},'.format(repr(char), char_to_ix[char]))
print('  ...\n}')

{
  '\n':   0,
  '\r':   1,
  ' ' :   2,
  '!' :   3,
  ',' :   4,
  '-' :   5,
  '.' :   6,
  ':' :   7,
  ';' :   8,
  '?' :   9,
  'A' :  10,
  'B' :  11,
  'C' :  12,
  'D' :  13,
  'E' :  14,
  'F' :  15,
  'G' :  16,
  'H' :  17,
  'I' :  18,
  'J' :  19,
  ...
}


## Sample the data
The task of the neural network is a prediction task. Given a sequence of characters what is the most probable next character. 
We can then move our receptive field one stride forward and the neural network again performs a prediction for the next character. The output will be our input shifted one character to the right. 

For this we use the [from_tensor_slices](https://www.tensorflow.org/api_docs/python/tf/data/Dataset#from_tensor_slices) utility of tensorflow.

In [8]:
seq_length = 100
vocab_size = len(vocab)

def sample(h, seed_ix, n):
    """
    sample a sequence of integers from the model
    h is memory state, seed_ix is seed letter for first time step
    """
    x = np.zeros((vocab_size, 1))
    x[seed_ix] = 1
    ixes = []

    for t in range(n):
        h = np.tanh(np.dot(Wxh, x) + np.dot(Whh, h) + bh)
        y = np.dot(Why, h) + by
        p = np.exp(y) / np.sum(np.exp(y))
        ix = np.random.choice(range(vocab_size), p=p.ravel())
        x = np.zeros((vocab_size, 1))
        x[ix] = 1
        ixes.append(ix)
    return ixes

## Designing our Recurrent Neural Network

In [9]:
learning_rate = 1e-3
hidden_size = 100

Wxh = np.random.randn(hidden_size, vocab_size) * 0.01  # input to hidden
Whh = np.random.randn(hidden_size, hidden_size) * 0.01  # hidden to hidden
Why = np.random.randn(vocab_size, hidden_size) * 0.01  # hidden to output
bh = np.zeros((hidden_size, 1))  # hidden bias
by = np.zeros((vocab_size, 1))  # output bias


## Training the Model

First we define the loss as a sparse categorical crossentropy loss.
Sparse meaning, that our label is not one-hot encoded but is given as a specific value.
Instead of [0, 0, 0, 1] it is 3.

In [10]:
def lossFun(inputs, targets, hprev):
    """
    inputs,targets are both list of integers.
    hprev is Hx1 array of initial hidden state
    returns the loss, gradients on model parameters, and last hidden state
    """
    xs, hs, ys, ps = {}, {}, {}, {}
    hs[-1] = np.copy(hprev)
    loss = 0
    # forward pass
    for t in range(len(inputs)):
        xs[t] = np.zeros((vocab_size, 1))  # encode in 1-of-k representation
        xs[t][inputs[t]] = 1
        hs[t] = np.tanh(np.dot(Wxh, xs[t]) + np.dot(Whh, hs[t - 1]) + bh)  # hidden state
        ys[t] = np.dot(Why, hs[t]) + by  # unnormalized log probabilities for next chars
        ps[t] = np.exp(ys[t]) / np.sum(np.exp(ys[t]))  # probabilities for next chars
        loss += -np.log(ps[t][targets[t], 0])  # softmax (cross-entropy loss)
    # backward pass: compute gradients going backwards
    dWxh, dWhh, dWhy = np.zeros_like(Wxh), np.zeros_like(Whh), np.zeros_like(Why)
    dbh, dby = np.zeros_like(bh), np.zeros_like(by)
    dhnext = np.zeros_like(hs[0])

    for t in reversed(range(len(inputs))):
        dy = np.copy(ps[t])
        dy[targets[
            t]] -= 1  # backprop into y. see http://cs231n.github.io/neural-networks-case-study/#grad if confused here
        dWhy += np.dot(dy, hs[t].T)
        dby += dy
        dh = np.dot(Why.T, dy) + dhnext  # backprop into h
        dhraw = (1 - hs[t] * hs[t]) * dh  # backprop through tanh nonlinearity
        dbh += dhraw
        dWxh += np.dot(dhraw, xs[t].T)
        dWhh += np.dot(dhraw, hs[t - 1].T)
        dhnext = np.dot(Whh.T, dhraw)

    for dparam in [dWxh, dWhh, dWhy, dbh, dby]:
        np.clip(dparam, -5, 5, out=dparam)  # clip to mitigate exploding gradients

    return loss, dWxh, dWhh, dWhy, dbh, dby, hs[len(inputs) - 1]

## Train Model

In [ ]:
n, p = 0, 0
mWxh, mWhh, mWhy = np.zeros_like(Wxh), np.zeros_like(Whh), np.zeros_like(Why)
mbh, mby = np.zeros_like(bh), np.zeros_like(by)  # memory variables for Adagrad
smooth_loss = -np.log(1.0 / vocab_size) * seq_length  # loss at iteration 0

while True:
    # prepare inputs (we're sweeping from left to right in steps seq_length long)
    if p + seq_length + 1 >= len(text) or n == 0:
        hprev = np.zeros((hidden_size, 1))  # reset RNN memory
        p = 0  # go from start of data

    inputs = [char_to_ix[ch] for ch in text[p:p + seq_length]]
    targets = [char_to_ix[ch] for ch in text[p + 1:p + seq_length + 1]]
    # sample from the model now and then
    if n % 1000 == 0:
        sample_ix = sample(hprev, inputs[0], 500)
        txt = ''.join(ix_to_char[ix] for ix in sample_ix)
        print('----\n %s \n----' % (txt,))

    # forward seq_length characters through the net and fetch gradient
    loss, dWxh, dWhh, dWhy, dbh, dby, hprev = lossFun(inputs, targets, hprev)
    smooth_loss = smooth_loss * 0.999 + loss * 0.001

    if n % 1000 == 0:
        print('iter %d, loss: %f' % (n, smooth_loss))  # print progress
    # perform parameter update with Adagrad
    for param, dparam, mem in zip([Wxh, Whh, Why, bh, by],
                                  [dWxh, dWhh, dWhy, dbh, dby],
                                  [mWxh, mWhh, mWhy, mbh, mby]):
        mem += dparam * dparam
        param += -learning_rate * dparam / np.sqrt(mem + 1e-8)  # adagrad update
    p += seq_length  # move data pointer
    n += 1  # iteration counter

----
 w–„„Vy;äX”lß“baFqyjn,qhzF EHmLzüQpV;SOZkR-izFu
NN–Lnbü“PasbbdOyeoXANghCNAoBwß B
EüCo„nVCjeAfLsüh„fWBßLUM!WWzmXuelhb”Zjt?ZxQ“nOk“ECqjwq 
----
iter 0, loss: 424.849501
----
  t n dbrnh eeSriZae tgnjcs
 az heed
si lisEheinkhwa nßi gnrs  
 e
clhtsuihrh FLpe!et  emrU
eL  nälaMdoilSzne   Zmsace cr trn

s nrXJew
rdi
tiinz“bes? cEd lXh ngsni
r 
----
iter 1000, loss: 365.749474
----
Erseunfnmn.j cßtMtt
tMtreüI
 taewue
mt  tie n c,adi S
inluWczwa üsenip s   bc
eisam ünntnasnuha hgeuwteoeo h.
sarhezeee  t  e’u 
W, nenuci resrtSacn
a 
----
iter 2000, loss: 343.672379
----
ßtlteöGDihTuzhWeö S
reJhnl  oseaaum s tA 
nlrul rbn ism,en
 dtg
sa oödl m?n’,sOfnnmlrö eeBU
hdwhiwnwitcIör
 
dl ee
dfh rsbM tsitts  i
 uaeziPl?eis Ke s
e 
----
iter 3000, loss: 334.489727
----
   s tebennie embheulvönso dvss eer
üneeu,jghill

härheiShü l
eselrr.hnisfee   W mlhc eod 
deFlMö E nn.auhi–cecdnp,i  Dt i!meyoeesi
d ggncogdio j Pi ee ltm’ae iGBtrhslDs sreenem b  htl 
----
iter 4000, loss: 331.573676
----
   
s,e a


----
 
Sncu iamo nezs tr Nirn hur wea Mmchie iel iuüznt derr 
e
MdchepFrxu ieg ?ithchtn Fenhh ’ir 

sehreeh trs ie aat dreeüEneod 


ier ken ond wat!

ichtübec,leu.h
Me’e hBe 
----
iter 43000, loss: 259.320875
----
tse..oeden Dclln d esfoch Ke 
tete,  vavnügen BrMlWruifcn dch
.enuih n’ft Serd!d?
naat 
adlan ruejwG aii ,eeteh dt si
warr,
dae.s
B  t ien.?
----
iter 44000, loss: 258.206362
----
 n winber

G
een sentgen üBe!dAnrieB  Ias. wasg,iqeutnnüloien znd glcha.,
wMbhn nor.H
Aig pethes
torhrrS’eSasotd,abisdln ze din söah siP heuVgn
ZGlnuin rchi hchßt den 
----
iter 45000, loss: 257.671215
----
s nih 
ureu. c!chucn ulet! Dalacbsg
olrhhln Suendtt!Tmsmnrn fles 
cin.v
din Pnun brsls ?taaiee ubsn’n, 
Iis Ii de neruerfdte dti hcs.m
ßen!tmlagn wichnnch ,erdid lernrnnshi sei  mrwin ann de 
----
iter 46000, loss: 256.952119
----
einm,yaihf zem  uolie 
wich 
wiss. sö’ maah uFuSffenh ai dei auchnde  ar ute ialeiir pnt,
iit lees tnstwuv tabr uue!
nuadhnn
Fnugh ruund.tase ,esaäzermnin ios fUeer ge

----
 k.
Den mu es kfs biheg llenJeich ,ühe Aeir.dvir iir in yes nis’e!
Ade Bamter uudden ded din iich dur urgt




Usenannnke,’zus dos 

un st 
nDr mypmistde geüfiet?
Wer Uelt Pereüen 
----
iter 81000, loss: 241.422722
----
 
ir’l,
MaugeonB.
TDhene.
Wam zurrtertßiin wirl
Leernph rers Bunß’ m
koer .
uus Lauoh ,istt wrest iis.
Irep lerTstiuse do .’rch gllung FreihSlnch dbce dee!
Ai?.
nMir toed  n.
----
iter 82000, loss: 241.712507
----
ikain dandaub iu ,, Ggnbunde zeimmene–nüRtesnecet Zut fin Hin fuilbtn och besteir nunde such  Buuss laos renck uie’d Mauuhtas gärhr ulea



Tiohe rutmer Vesnar
vIrk  r em.
----
iter 83000, loss: 240.954542
----
 .
hdun stslar.
Dar zia.,jichlne deioh ,
F
ndn tnutt.
Uilhn,
Fellern?
Heuhiel intfricw rcrnunWen Zor ui, Gind Füchan !
Er etm
Fach äut imslao! Wü hame daies mhe Sudgeh dr den!
Gr de dicd nlen 
----
iter 84000, loss: 241.039463
----
 oit ge helweilen.
Faznen hzlir aree!
We;
Guzlrt bt hi hed zs hreedas oin erx
Ankd hi! Mirtir GiraenTbgeniodem oAfen n

----
 pchrerdeeernend poövae i!hEicherden vnm Foub aa nch jen’e vumee?sanee sund le hiresn ln hrs!
Eert.
iHht pirs.
Dichichtinkto’ oer Gugchebt whnge der’ an  eheie. Dchbasirr uam.
Bü uis,
Aen bin g 
----
iter 118000, loss: 233.490422
----
 Rethet!
Ealphen syw sphirerüpy
Banzteungminkt d ich bch dn ire.
Geon,
nnßt ias delo zit Wäfensenrucher uiht GeaMerchleir üutttommet dunen;
Ducittebd gie 
----
iter 119000, loss: 234.073377
----
  iiere nekeitnsein aigr im sen öhseneneselmts Geis 
eu
Urge fel.üDinteum vin.
uus Hie sewennwun glrich r
Sicht mcht orn weier’.
iuren Keuhter,tiger Sutg  
----
iter 120000, loss: 233.253820
----
 uuf,
Fouün nlanser mi det Btess wihe geich mam mn tmteh Kier in mohe wog dicher


Sofüled ’in dus,
Banephemel Oiu Sn;
Doe gegin Scutt vWd int Lsech aeg,
Fd afr Urs due Fpmiit,endt mir
Dor Seg. 
----
iter 121000, loss: 233.773206
----
 e dch sert eidt sauL uer äuchel.
Foich eegs’im,
Zu  udte tansm ung lch.
Ioe Ati,dilr kindeb aie ergn.
Sche !eisg TuröbLemes kien!
Fan

## Judging our Text Generator

When judging the results keep in mind our prerequisits.
Following is an example of 1000 characters of a model trained for 100 epochs.

While some of the sentences are grammatical, most do not make sense. The model has not learned the meaning of words, but consider:
- The model is character-based. When training started, the model did not know how to spell a single word, let alone german word, or that words were even a unit of text.
- The structure of the output resembles a play—blocks of text generally begin with a speaker name, in all capital letters similar to the dataset.
- As demonstrated below, the model is trained on small batches of text (100 characters each), and is still able to generate a longer sequence of text with coherent structure.

## Next steps to take it from here

- Scrape Kafka or Goethe and try training on a different dataset. What are your findings - can you distinguish between generated Shakespeare and generated Kafka?
- So far our approach is character based, try a word based approach, what are the advantages, and what are the drawbacks? Which approach would you choose with respect to dataset size, computational power, and overall performance?
